In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler
import cantera as ct
import os
import gym 
import numpy as np 
from stable_baselines.common.policies import MlpPolicy, CnnLstmPolicy,MlpLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common import set_global_seeds
from stable_baselines import ACKTR, PPO2
from envs.sim_env import SimEnv
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


```python
 def _train_step(self, learning_rate, cliprange, obs, returns, masks, actions, values, neglogpacs, update,
                    writer, states=None, cliprange_vf=None):
        """
        Training of PPO2 Algorithm
        :param learning_rate: (float) learning rate
        :param cliprange: (float) Clipping factor
        :param obs: (np.ndarray) The current observation of the environment
        :param returns: (np.ndarray) the rewards
        :param masks: (np.ndarray) The last masks for done episodes (used in recurent policies)
        :param actions: (np.ndarray) the actions
        :param values: (np.ndarray) the values
        :param neglogpacs: (np.ndarray) Negative Log-likelihood probability of Actions
        :param update: (int) the current step iteration
        :param writer: (TensorFlow Summary.writer) the writer for tensorboard
        :param states: (np.ndarray) For recurrent policies, the internal state of the recurrent model
        :return: policy gradient loss, value function loss, policy entropy,
                approximation of kl divergence, updated clipping range, training update operation
        :param cliprange_vf: (float) Clipping factor for the value function
        """
        advs = returns - values
        advs = (advs - advs.mean()) / (advs.std() + 1e-8)
        td_map = {self.train_model.obs_ph: obs, self.action_ph: actions,
                  self.advs_ph: advs, self.rewards_ph: returns,
                  self.learning_rate_ph: learning_rate, self.clip_range_ph: cliprange,
                  self.old_neglog_pac_ph: neglogpacs, self.old_vpred_ph: values}
        if states is not None:
            td_map[self.train_model.states_ph] = states
            td_map[self.train_model.dones_ph] = masks

        if cliprange_vf is not None and cliprange_vf >= 0:
            td_map[self.clip_range_vf_ph] = cliprange_vf

        if states is None:
            update_fac = self.n_batch // self.nminibatches // self.noptepochs + 1
        else:
            update_fac = self.n_batch // self.nminibatches // self.noptepochs // self.n_steps + 1

        if writer is not None:
            # run loss backprop with summary, but once every 10 runs save the metadata (memory, compute time, ...)
            if self.full_tensorboard_log and (1 + update) % 10 == 0:
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, policy_loss, value_loss, policy_entropy, approxkl, clipfrac, _ = self.sess.run(
                    [self.summary, self.pg_loss, self.vf_loss, self.entropy, self.approxkl, self.clipfrac, self._train],
                    td_map, options=run_options, run_metadata=run_metadata)
                writer.add_run_metadata(run_metadata, 'step%d' % (update * update_fac))
            else:
                summary, policy_loss, value_loss, policy_entropy, approxkl, clipfrac, _ = self.sess.run(
                    [self.summary, self.pg_loss, self.vf_loss, self.entropy, self.approxkl, self.clipfrac, self._train],
                    td_map)
            writer.add_summary(summary, (update * update_fac))
        else:
            policy_loss, value_loss, policy_entropy, approxkl, clipfrac, _ = self.sess.run(
                [self.pg_loss, self.vf_loss, self.entropy, self.approxkl, self.clipfrac, self._train], td_map)

        return policy_loss, value_loss, policy_entropy, approxkl, clipfrac
```

In [3]:
env = SubprocVecEnv([SimEnv])
model = PPO2(MlpLstmPolicy, env, gamma=0.99, n_steps=128, ent_coef=0.01, learning_rate=2.5e-4, vf_coef=0.5,
                 max_grad_norm=0.5, lam=0.95, nminibatches=1, noptepochs=4, cliprange=0.2, cliprange_vf=None,
                 verbose=0, tensorboard_log=None, _init_setup_model=True, policy_kwargs=None,
                 full_tensorboard_log=False)
model.learn(total_timesteps=50000)





Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [13]:
from stable_baselines.common.vec_env import DummyVecEnv
env = DummyVecEnv([SimEnv])
obs = env.reset()

In [14]:
game_over = False
while not game_over: 
    action, _states = model.predict(obs)
    obs, rewards, game_over, info = env.step(action)
    env.render(mode='human')

1	 0.00	 1446.86	 -0.00	 0.06	 1.06	 245.82	 -45.71	5.00	286.53
2	 0.00	 1465.42	 -0.00	 0.06	 1.15	 248.67	 -40.00	5.00	283.68
3	 0.00	 1102.66	 0.80	 0.04	 0.68	 61.58	 -243.35	5.00	299.93
4	 0.00	 1168.58	 0.75	 0.04	 0.78	 111.69	 -189.91	5.00	296.61
5	 0.00	 1176.01	 0.74	 0.04	 0.79	 116.79	 -184.41	5.00	296.20
6	 0.01	 1038.52	 0.85	 0.03	 0.60	 4.23	 -303.73	5.00	302.97
7	 0.01	 954.44	 0.89	 0.03	 0.49	 -84.75	 -396.48	5.00	306.74
8	 0.01	 1002.93	 0.87	 0.03	 0.55	 -31.38	 -341.04	5.00	304.67
9	 0.01	 886.92	 0.91	 0.02	 0.42	 -168.41	 -482.99	5.00	309.59
10	 0.01	 917.22	 0.90	 0.02	 0.45	 -129.40	 -442.80	5.00	308.41
11	 0.01	 917.25	 0.90	 0.02	 0.45	 -129.35	 -442.76	5.00	308.42
12	 0.01	 885.97	 0.91	 0.02	 0.41	 -169.58	 -484.29	5.00	309.72
13	 0.01	 901.66	 0.91	 0.02	 0.43	 -149.05	 -463.15	5.00	309.11
14	 0.01	 867.08	 0.92	 0.02	 0.39	 -195.07	 -510.55	5.00	310.50
15	 0.02	 865.94	 0.92	 0.02	 0.39	 -196.62	 -512.17	5.00	310.56
16	 0.02	 862.26	 0.92	 0.02	 0.38	 -2

130	 0.13	 859.28	 0.92	 0.02	 0.35	 -204.75	 -521.66	5.00	312.04
131	 0.13	 856.70	 0.92	 0.02	 0.35	 -208.37	 -525.37	5.00	312.12
132	 0.13	 858.57	 0.92	 0.02	 0.35	 -205.75	 -522.69	5.00	312.07
133	 0.13	 854.43	 0.92	 0.02	 0.34	 -211.56	 -528.65	5.00	312.22
134	 0.13	 856.24	 0.92	 0.02	 0.34	 -209.00	 -526.03	5.00	312.17
135	 0.13	 853.94	 0.92	 0.02	 0.34	 -212.21	 -529.36	5.00	312.28
136	 0.14	 858.25	 0.92	 0.02	 0.35	 -206.12	 -523.14	5.00	312.16
137	 0.14	 856.59	 0.92	 0.02	 0.34	 -208.46	 -525.54	5.00	312.22
138	 0.14	 858.57	 0.92	 0.02	 0.35	 -205.66	 -522.68	5.00	312.16
139	 0.14	 857.74	 0.92	 0.02	 0.34	 -206.80	 -523.87	5.00	312.21
140	 0.14	 858.46	 0.92	 0.02	 0.34	 -205.80	 -522.84	5.00	312.18
141	 0.14	 859.86	 0.91	 0.02	 0.35	 -203.82	 -520.82	5.00	312.14
142	 0.14	 860.55	 0.91	 0.02	 0.35	 -202.87	 -519.85	5.00	312.12
143	 0.14	 861.48	 0.91	 0.02	 0.35	 -201.56	 -518.52	5.00	312.10
144	 0.14	 863.76	 0.91	 0.02	 0.35	 -198.38	 -515.26	5.00	312.03
145	 0.14	

264	 0.26	 877.50	 0.91	 0.02	 0.33	 -178.31	 -495.95	5.00	312.90
265	 0.26	 878.56	 0.91	 0.02	 0.33	 -176.86	 -494.48	5.00	312.88
266	 0.27	 878.64	 0.91	 0.02	 0.33	 -176.75	 -494.37	5.00	312.88
267	 0.27	 879.05	 0.91	 0.02	 0.33	 -176.19	 -493.80	5.00	312.87
268	 0.27	 877.34	 0.91	 0.02	 0.32	 -178.51	 -496.18	5.00	312.93
269	 0.27	 878.10	 0.91	 0.02	 0.32	 -177.46	 -495.12	5.00	312.92
270	 0.27	 879.24	 0.91	 0.02	 0.33	 -175.91	 -493.53	5.00	312.90
271	 0.27	 880.86	 0.91	 0.02	 0.33	 -173.71	 -491.30	5.00	312.86
272	 0.27	 878.80	 0.91	 0.02	 0.32	 -176.48	 -494.15	5.00	312.94
273	 0.27	 880.39	 0.91	 0.02	 0.32	 -174.32	 -491.95	5.00	312.91
274	 0.27	 881.68	 0.91	 0.02	 0.33	 -172.55	 -490.16	5.00	312.89
275	 0.28	 882.29	 0.91	 0.02	 0.33	 -171.74	 -489.33	5.00	312.87
276	 0.28	 882.29	 0.91	 0.02	 0.33	 -171.74	 -489.33	5.00	312.87
277	 0.28	 880.58	 0.91	 0.02	 0.32	 -174.03	 -491.68	5.00	312.93
278	 0.28	 881.96	 0.91	 0.02	 0.32	 -172.17	 -489.79	5.00	312.90
279	 0.28	

391	 0.39	 893.41	 0.91	 0.02	 0.31	 -156.00	 -474.18	5.00	313.57
392	 0.39	 892.80	 0.91	 0.02	 0.31	 -156.80	 -475.01	5.00	313.60
393	 0.39	 890.81	 0.91	 0.02	 0.30	 -159.43	 -477.69	5.00	313.66
394	 0.39	 891.94	 0.91	 0.02	 0.30	 -157.91	 -476.16	5.00	313.64
395	 0.40	 890.53	 0.91	 0.02	 0.30	 -159.79	 -478.08	5.00	313.68
396	 0.40	 891.46	 0.91	 0.02	 0.30	 -158.54	 -476.81	5.00	313.68
397	 0.40	 892.75	 0.91	 0.02	 0.30	 -156.81	 -475.07	5.00	313.66
398	 0.40	 894.05	 0.91	 0.02	 0.30	 -155.07	 -473.32	5.00	313.64
399	 0.40	 893.03	 0.91	 0.02	 0.30	 -156.43	 -474.70	5.00	313.67
400	 0.40	 892.78	 0.91	 0.02	 0.30	 -156.77	 -475.04	5.00	313.67
401	 0.40	 892.96	 0.91	 0.02	 0.30	 -156.52	 -474.79	5.00	313.67
402	 0.40	 893.04	 0.91	 0.02	 0.30	 -156.40	 -474.69	5.00	313.69
403	 0.40	 894.25	 0.90	 0.02	 0.30	 -154.79	 -473.06	5.00	313.67
404	 0.40	 894.77	 0.90	 0.02	 0.30	 -154.10	 -472.36	5.00	313.66
405	 0.41	 894.70	 0.90	 0.02	 0.30	 -154.20	 -472.45	5.00	313.66
406	 0.41	

523	 0.52	 897.45	 0.90	 0.02	 0.28	 -149.70	 -468.76	5.00	314.59
524	 0.52	 897.45	 0.90	 0.02	 0.28	 -149.68	 -468.76	5.00	314.60
525	 0.53	 896.84	 0.90	 0.02	 0.28	 -150.48	 -469.57	5.00	314.62
526	 0.53	 895.49	 0.91	 0.02	 0.28	 -152.24	 -471.38	5.00	314.66
527	 0.53	 894.81	 0.91	 0.02	 0.28	 -153.14	 -472.30	5.00	314.69
528	 0.53	 896.07	 0.91	 0.02	 0.28	 -151.46	 -470.61	5.00	314.67
529	 0.53	 896.39	 0.91	 0.02	 0.28	 -151.04	 -470.18	5.00	314.67
530	 0.53	 895.92	 0.91	 0.02	 0.28	 -151.66	 -470.81	5.00	314.68
531	 0.53	 896.14	 0.91	 0.02	 0.28	 -151.36	 -470.52	5.00	314.69
532	 0.53	 896.71	 0.91	 0.02	 0.28	 -150.59	 -469.75	5.00	314.68
533	 0.53	 896.82	 0.91	 0.02	 0.28	 -150.46	 -469.61	5.00	314.68
534	 0.53	 897.45	 0.90	 0.02	 0.28	 -149.61	 -468.75	5.00	314.68
535	 0.54	 898.27	 0.90	 0.02	 0.28	 -148.52	 -467.66	5.00	314.67
536	 0.54	 898.02	 0.90	 0.02	 0.28	 -148.85	 -468.00	5.00	314.68
537	 0.54	 898.59	 0.90	 0.02	 0.28	 -148.10	 -467.24	5.00	314.68
538	 0.54	

...

15469	 15.47	 898.07	 0.90	 0.02	 0.09	 -156.83	 -467.92	5.00	321.57
15470	 15.47	 898.09	 0.90	 0.02	 0.09	 -156.81	 -467.90	5.00	321.57
15471	 15.47	 898.09	 0.90	 0.02	 0.09	 -156.81	 -467.90	5.00	321.57
15472	 15.47	 898.09	 0.90	 0.02	 0.09	 -156.81	 -467.91	5.00	321.57
15473	 15.47	 898.13	 0.90	 0.02	 0.09	 -156.76	 -467.85	5.00	321.56
15474	 15.47	 898.14	 0.90	 0.02	 0.09	 -156.74	 -467.83	5.00	321.56
15475	 15.47	 898.15	 0.90	 0.02	 0.09	 -156.74	 -467.83	5.00	321.56
15476	 15.48	 898.15	 0.90	 0.02	 0.09	 -156.74	 -467.83	5.00	321.56
15477	 15.48	 898.15	 0.90	 0.02	 0.09	 -156.73	 -467.82	5.00	321.56
15478	 15.48	 898.16	 0.90	 0.02	 0.09	 -156.72	 -467.80	5.00	321.56
15479	 15.48	 898.18	 0.90	 0.02	 0.09	 -156.70	 -467.78	5.00	321.56
15480	 15.48	 898.21	 0.90	 0.02	 0.09	 -156.66	 -467.74	5.00	321.56
15481	 15.48	 898.22	 0.90	 0.02	 0.09	 -156.65	 -467.73	5.00	321.56
15482	 15.48	 898.23	 0.90	 0.02	 0.09	 -156.63	 -467.71	5.00	321.56
15483	 15.48	 898.23	 0.90	 0.02	 

15590	 15.59	 898.09	 0.90	 0.02	 0.09	 -156.94	 -467.91	5.00	321.55
15591	 15.59	 898.09	 0.90	 0.02	 0.09	 -156.94	 -467.91	5.00	321.55
15592	 15.59	 898.10	 0.90	 0.02	 0.09	 -156.93	 -467.89	5.00	321.55
15593	 15.59	 898.10	 0.90	 0.02	 0.09	 -156.93	 -467.89	5.00	321.55
15594	 15.59	 898.11	 0.90	 0.02	 0.09	 -156.91	 -467.87	5.00	321.55
15595	 15.59	 898.07	 0.90	 0.02	 0.09	 -156.97	 -467.93	5.00	321.55
15596	 15.60	 898.10	 0.90	 0.02	 0.09	 -156.93	 -467.88	5.00	321.55
15597	 15.60	 898.11	 0.90	 0.02	 0.09	 -156.91	 -467.87	5.00	321.55
15598	 15.60	 898.14	 0.90	 0.02	 0.09	 -156.88	 -467.83	5.00	321.55
15599	 15.60	 898.11	 0.90	 0.02	 0.09	 -156.92	 -467.87	5.00	321.55
15600	 15.60	 898.11	 0.90	 0.02	 0.09	 -156.93	 -467.88	5.00	321.55
15601	 15.60	 898.07	 0.90	 0.02	 0.09	 -156.97	 -467.92	5.00	321.55
15602	 15.60	 898.10	 0.90	 0.02	 0.09	 -156.94	 -467.89	5.00	321.55
15603	 15.60	 898.11	 0.90	 0.02	 0.09	 -156.93	 -467.88	5.00	321.55
15604	 15.60	 898.13	 0.90	 0.02	 

15715	 15.71	 898.25	 0.90	 0.02	 0.09	 -156.86	 -467.69	5.00	321.54
15716	 15.72	 898.25	 0.90	 0.02	 0.09	 -156.86	 -467.69	5.00	321.54
15717	 15.72	 898.26	 0.90	 0.02	 0.09	 -156.85	 -467.68	5.00	321.54
15718	 15.72	 898.25	 0.90	 0.02	 0.09	 -156.87	 -467.69	5.00	321.54
15719	 15.72	 898.26	 0.90	 0.02	 0.09	 -156.86	 -467.68	5.00	321.54
15720	 15.72	 898.25	 0.90	 0.02	 0.09	 -156.86	 -467.68	5.00	321.54
15721	 15.72	 898.28	 0.90	 0.02	 0.09	 -156.83	 -467.65	5.00	321.54
15722	 15.72	 898.21	 0.90	 0.02	 0.09	 -156.91	 -467.74	5.00	321.54
15723	 15.72	 898.23	 0.90	 0.02	 0.09	 -156.89	 -467.72	5.00	321.54
15724	 15.72	 898.21	 0.90	 0.02	 0.09	 -156.93	 -467.75	5.00	321.54
15725	 15.72	 898.23	 0.90	 0.02	 0.09	 -156.89	 -467.71	5.00	321.54
15726	 15.73	 898.23	 0.90	 0.02	 0.09	 -156.90	 -467.71	5.00	321.54
15727	 15.73	 898.20	 0.90	 0.02	 0.09	 -156.93	 -467.75	5.00	321.54
15728	 15.73	 898.20	 0.90	 0.02	 0.09	 -156.93	 -467.75	5.00	321.54
15729	 15.73	 898.21	 0.90	 0.02	 

15837	 15.84	 898.07	 0.90	 0.02	 0.09	 -157.23	 -467.93	5.00	321.54
15838	 15.84	 898.07	 0.90	 0.02	 0.09	 -157.23	 -467.92	5.00	321.54
15839	 15.84	 898.10	 0.90	 0.02	 0.09	 -157.19	 -467.89	5.00	321.53
15840	 15.84	 898.04	 0.90	 0.02	 0.09	 -157.27	 -467.96	5.00	321.54
15841	 15.84	 898.07	 0.90	 0.02	 0.09	 -157.24	 -467.93	5.00	321.53
15842	 15.84	 898.00	 0.90	 0.02	 0.09	 -157.33	 -468.02	5.00	321.54
15843	 15.84	 897.99	 0.90	 0.02	 0.09	 -157.34	 -468.03	5.00	321.54
15844	 15.84	 898.03	 0.90	 0.02	 0.09	 -157.29	 -467.99	5.00	321.54
15845	 15.84	 898.03	 0.90	 0.02	 0.09	 -157.29	 -467.98	5.00	321.53
15846	 15.85	 898.07	 0.90	 0.02	 0.09	 -157.24	 -467.93	5.00	321.53
15847	 15.85	 898.09	 0.90	 0.02	 0.09	 -157.22	 -467.90	5.00	321.53
15848	 15.85	 898.09	 0.90	 0.02	 0.09	 -157.21	 -467.90	5.00	321.53
15849	 15.85	 898.09	 0.90	 0.02	 0.09	 -157.22	 -467.91	5.00	321.53
15850	 15.85	 898.11	 0.90	 0.02	 0.09	 -157.19	 -467.87	5.00	321.53
15851	 15.85	 898.10	 0.90	 0.02	 

15962	 15.96	 898.18	 0.90	 0.02	 0.09	 -157.22	 -467.78	5.00	321.53
15963	 15.96	 898.19	 0.90	 0.02	 0.09	 -157.21	 -467.77	5.00	321.53
15964	 15.96	 898.19	 0.90	 0.02	 0.09	 -157.21	 -467.77	5.00	321.52
15965	 15.96	 898.20	 0.90	 0.02	 0.09	 -157.19	 -467.75	5.00	321.52
15966	 15.97	 898.23	 0.90	 0.02	 0.09	 -157.16	 -467.72	5.00	321.52
15967	 15.97	 898.23	 0.90	 0.02	 0.09	 -157.16	 -467.72	5.00	321.52
15968	 15.97	 898.25	 0.90	 0.02	 0.09	 -157.14	 -467.69	5.00	321.52
15969	 15.97	 898.25	 0.90	 0.02	 0.09	 -157.13	 -467.69	5.00	321.52
15970	 15.97	 898.28	 0.90	 0.02	 0.09	 -157.09	 -467.65	5.00	321.52
15971	 15.97	 898.25	 0.90	 0.02	 0.09	 -157.13	 -467.69	5.00	321.52
15972	 15.97	 898.27	 0.90	 0.02	 0.09	 -157.12	 -467.67	5.00	321.52
15973	 15.97	 898.28	 0.90	 0.02	 0.09	 -157.10	 -467.65	5.00	321.52
15974	 15.97	 898.29	 0.90	 0.02	 0.09	 -157.09	 -467.64	5.00	321.52
15975	 15.97	 898.26	 0.90	 0.02	 0.09	 -157.13	 -467.68	5.00	321.52
15976	 15.98	 898.27	 0.90	 0.02	 

In [12]:
%debug

> /home/edwin/anaconda3/envs/tf1_env/lib/python3.7/site-packages/stable_baselines/common/tile_images.py(15)tile_images()
     13     """
     14     img_nhwc = np.asarray(img_nhwc)
---> 15     n_images, height, width, n_channels = img_nhwc.shape
     16     # new_height was named H before
     17     new_height = int(np.ceil(np.sqrt(n_images)))

ipdb> img_nhwc
array([None], dtype=object)
ipdb> u
> /home/edwin/anaconda3/envs/tf1_env/lib/python3.7/site-packages/stable_baselines/common/vec_env/subproc_vec_env.py(134)render()
    132         imgs = [pipe.recv() for pipe in self.remotes]
    133         # Create a big image by tiling images from subprocesses
--> 134         bigimg = tile_images(imgs)
    135         if mode == 'human':
    136             import cv2

ipdb> self
ipdb> self.remotes
(<multiprocessing.connection.Connection object at 0x7f734262da50>,)
ipdb> q
